# Interpretable Machine Learning
## Exercise Sheet: 6
## This exercise sheet covers chapters 8.6 and 9.1 from the IML book by Christoph Molnar

Kristin Blesch (blesch@leibniz-bips.de)<br>
Niklas Koenen (koenen@leibniz-bips.de)
<hr style="border:1.5px solid gray"> </hr>

# 1) Global Surrogate

**a)** Explain the method 'Global Surrogate' and give an example.  

**Solution:**

With a global surrogate, interpretability comes from creating an intrinsic copy of the actual model. No new model is fitted on the training data $(X, Y)$, but an intrinsically interpretable model learns the behavior of the given model $\hat{f}$, i.e. the actual outcome $Y$ is replaced by the model output $\hat{Y} = \hat{f}(X)$ and thus an intrinsic model is fitted on $(X, \hat{Y})$, which by definition can be interpreted.  
For example, a linear model or a decision tree can be used to learn the predictions of a neural network.

**b)** How can we measure the performance of a global surrogate. What is the consequence if the model performs very bad but the surrogate very good?  

**Solution:**

One way to measure how well the surrogate replicates the black box model is the R² measure:
$$
R^2=1-\frac{SSE}{SST}=1-\frac{\sum_{i=1}^n(\hat{y}_*^{(i)}-\hat{y}^{(i)})^2}{\sum_{i=1}^n(\hat{y}^{(i)}-\bar{\hat{y}})^2}
$$
The performance of the black box model does not play a role in training the surrogate model. The interpretation of the surrogate model is still valid because it makes statements about the model and not about the real world. But of course the interpretation of the surrogate model becomes irrelevant if the black box model is bad, because then the black box model itself is irrelevant.

# 2) Individual Conditional Expectation (ICE)

**a)** What is an individual conditional expectation (ICE) plot and what is its relation to the PDP? Also explain the variants c-ICE and d-ICE and their advantages.

**Solution:**

Individual Conditional Expectation (ICE) plots visualise one line per instance that shows how the instance’s prediction changes when a feature changes. This is therefore a local equivalent of the global PDP plots. Instead of calculating the mean of the lines of all instances, they are plotted one by one, i.e. an ICE plot visualizes the dependence of the prediction on a feature for each instance separately, resulting in one line per instance, compared to one line overall in partial dependence plots.  

**centered ICE (c-ICE):** It is sometimes hard to compare the individual ICE lines because they always start at different values, i.e. they are shifted on the y-axis differently. A solution is provided by the c-ICE plot, which anchors the curves at the lower end of the feature: ($x^a$ anchor point)
$$
\hat{f}_{cent}^{(i)}=\hat{f}^{(i)}-\mathbf{1}\hat{f}(x^{a},x^{(i)}_{C})
$$

**derivative ICE (d-ICE):** To spot heterogeneity in a visually easier way, the individual derivatives of the prediction function of the corresponding feature can be considered. Furthermore, interactions can be revealed, since the derivatives must be constant in the case of no interactions.

**b)** We again consider the example from Sheet 4 Task 1, but in this case we train a decision tree via gradient boosting. Execute the following code, which is almost identical to the one in sheet 4, only with gradient boosting:

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# Get data
cal_housing = fetch_california_housing()
X = pd.DataFrame(cal_housing.data, columns=cal_housing.feature_names)
y = cal_housing.target

# Get train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Train model and calculate R²-score
from sklearn.ensemble import HistGradientBoostingRegressor

model = HistGradientBoostingRegressor(random_state = 42)
model.fit(X_train, y_train)
print(f"Test R2 score: {model.score(X_test, y_test):.2f}")

Now create the ICE plot for each feature using the already known function [`PartialDependenceDisplay`](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.PartialDependenceDisplay.html#sklearn-inspection-partialdependencedisplay).  

**Solution:**

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
fig, ax = plt.subplots(nrows = 2, ncols = 4,figsize=(24, 12))

display = PartialDependenceDisplay.from_estimator(
    model,
    X_test,
    ["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup", "Latitude", "Longitude"],
    kind="both",
    ice_lines_kw={"color": "tab:blue", "alpha": 0.1, "linewidth": 0.5},
    pd_line_kw={"color": "tab:orange", "linestyle": "--", "linewidth": 3},
    ax = ax
)
plt.show()

**Bonus:** Modify the predict function of the trained model to produce a c-ICE for the feature `MedInc`.

In [ ]:
# Save model predict function
predict_fun = model.predict

In [ ]:
# Create the normalizing constant for each ICE line
X = X_test.copy()
q_5 = X.quantile(0.05)[0]
X.MedInc = q_5
normalize_constant = predict_fun(X)

# Define new predict function
def new_predict(X):
  pred = predict_fun(X) - normalize_constant
  
  return pred

# Set the new predict function in our model and calculate again the ICE plot for feature 'MedInc'
model.predict = new_predict
display = PartialDependenceDisplay.from_estimator(
    model,
    X_test,
    ["MedInc"],
    kind="both",
    ice_lines_kw={"color": "tab:blue", "alpha": 0.1, "linewidth": 0.5},
    pd_line_kw={"color": "tab:orange", "linestyle": "--", "linewidth": 3}
)
plt.show()